In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
human_df = pd.read_table('human_data.txt')

In [50]:
human_df.head()

,sequence,class
0,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...,4
1,ATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG...,4
2,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
3,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
4,ATGCAACAGCATTTTGAATTTGAATACCAGACCAAAGTGGATGGTG...,3


In [51]:
#default size 6 
def getkmers(sequence, size = 6):
    return [sequence[x: x+size].lower() for x in range(len(sequence) - size + 1)]

In [52]:
human_df['words'] = human_df.apply(lambda x: getkmers(x['sequence']), axis = 1)
human_df = human_df.drop('sequence' , axis = 1)


In [53]:
human_df.head()

,class,words
0,4,"[atgccc, tgcccc, gcccca, ccccaa, cccaac, ccaac..."
1,4,"[atgaac, tgaacg, gaacga, aacgaa, acgaaa, cgaaa..."
2,3,"[atgtgt, tgtgtg, gtgtgg, tgtggc, gtggca, tggca..."
3,3,"[atgtgt, tgtgtg, gtgtgg, tgtggc, gtggca, tggca..."
4,3,"[atgcaa, tgcaac, gcaaca, caacag, aacagc, acagc..."


In [54]:
human_text = list(human_df['words'])
for item in range(len(human_text)):
    human_text[item] = ' '.join(human_text[item])

In [55]:
y = human_df.iloc[:,0].values

In [56]:
print(human_text[0])

atgccc tgcccc gcccca ccccaa cccaac ccaact caacta aactaa actaaa ctaaat taaata aaatac aatact atacta tactac actacc ctaccg taccgt accgta ccgtat cgtatg gtatgg tatggc atggcc tggccc ggccca gcccac cccacc ccacca caccat accata ccataa cataat ataatt taatta aattac attacc ttaccc tacccc accccc ccccca ccccat cccata ccatac catact atactc tactcc actcct ctcctt tcctta ccttac cttaca ttacac tacact acacta cactat actatt ctattc tattcc attcct ttcctc tcctca cctcat ctcatc tcatca catcac atcacc tcaccc caccca acccaa cccaac ccaact caacta aactaa actaaa ctaaaa taaaaa aaaaat aaaata aaatat aatatt atatta tattaa attaaa ttaaac taaaca aaacac aacaca acacaa cacaaa acaaac caaact aaacta aactac actacc ctacca taccac accacc ccacct caccta acctac cctacc ctacct tacctc acctcc cctccc ctccct tccctc ccctca cctcac ctcacc tcacca caccaa accaaa ccaaag caaagc aaagcc aagccc agccca gcccat cccata ccataa cataaa ataaaa taaaaa aaaaat aaaata aaataa aataaa ataaaa taaaaa aaaaaa aaaaat aaaatt aaatta aattat attata ttataa tataac ataaca taacaa aacaaa acaaac

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range = (4,4))
x = cv.fit_transform(human_text)


In [58]:
print(x.shape)

(4380, 232414)


In [59]:
human_df['class'].value_counts().sort_index()

0     531
1     534
2     349
3     672
4     711
5     240
6    1343
Name: class, dtype: int64

In [60]:
x

<4380x232414 sparse matrix of type '<class 'numpy.int64'>'
	with 5406441 stored elements in Compressed Sparse Row format>

In [61]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y ,test_size=0.2 , random_state=2)

In [62]:
print(x.shape , x_train.shape,x_test.shape)

(4380, 232414) (3504, 232414) (876, 232414)


In [63]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha = 0.1)
classifier.fit(x_train, y_train)

MultinomialNB(alpha=0.1)

In [64]:
y_pred = classifier.predict(x_test)

In [65]:
from sklearn.metrics import accuracy_score , f1_score, precision_score, recall_score

print('Confusion Matrix \n')
print(pd.crosstab(pd.Series(y_test, name= 'Actual'), pd.Series(y_pred, name='Predicted')))

def get_metrics(y_test,y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average = 'weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 =f1_score(y_test, y_predicted, average = 'weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)

print(" Accuravy = %.3f \n precision = %.3f \n recall = %.3f \n f1 = %.3f" % (accuracy, precision, recall, f1))

Confusion Matrix 

Predicted    0   1   2    3    4   5    6
Actual                                   
0          108   0   0    0    0   0    4
1            0  96   0    0    0   0    0
2            0   0  60    1    0   0    2
3            0   0   0  129    0   0    1
4            1   0   0    0  124   0    2
5            2   0   0    0    0  49    2
6            0   0   1    0    0   0  294
 Accuravy = 0.982 
 precision = 0.982 
 recall = 0.982 
 f1 = 0.982
